In [1]:
import os
import pandas
from pandas import read_csv
import json
import pickle
from datetime import datetime
import sys
import PyPDF2 as pypdf
import requests

## Parse the data dump

Note that this code still needs a downloader to pull data from either github or google drive

In [ ]:
#### Create curatedBy Object
def generate_curator():
    todate = datetime.now()
    curatedByObject = {"@type": "Organization", "identifier": "covid19LST", "url": "https://www.covid19lst.org/", 
                              "name": "COVID-19 Literature Surveillance Team", "affiliation": "", 
                              "curationDate": todate.strftime("%Y-%m-%d")}
    return(curatedByObject)

In [ ]:
def generate_author():
    authorObject = generate_curator()
    authorObject.pop('curationDate')
    memberlist = read_csv('data/LST members.txt',delimiter='\t',header=0,encoding='UTF-8')
    memberlist.rename(columns={'affiliation':'affiliation list'}, inplace=True)
    memberlist['affiliation']='blank'
    for i in range(len(memberlist)):
        affiliationlist = memberlist.iloc[i]['affiliation list'].split(';')
        tmplist = []
        for eachaffiliation in affiliationlist:
            tmplist.append({"name":eachaffiliation})
        memberlist.at[i,'affiliation'] = tmplist
    memberlist.drop(columns='affiliation list',inplace=True)
    memberdictlist = memberlist.to_dict('records')
    authorObject['members']=memberdictlist 
    return(authorObject)

In [ ]:
def fix_keywords(keywordstring):
    if keywordstring != keywordstring: ## Is it Nan?
        keywordlist = []
    elif keywordstring =="": ## Is it an empty string?
        keywordlist = []
    elif keywordstring == None: ## Is there no keywordstring?
        keywordlist = []
    else:
        keywordlist = keywordstring.lstrip('[').rstrip(']').replace('"','').split(',')
    return(keywordlist)

In [ ]:
def generate_lst_dump(datadmp):
    cleandata = []
    authorObject = generate_author()
    authorObject.pop('members')
    datadmp['_id'] = 'pmid'+datadmp['PMID'].astype(str)  
    for i in range(len(datadmp)):
        keywordlist = fix_keywords(datadmp.iloc[i]['Topics'])
        tmpdict={'_id':datadmp.iloc[i]['_id'],'keywords':keywordlist,
                 'covid19LST':{'@type':'Rating',
                                 'ratingExplanation':datadmp.iloc[i]['Methodology'],
                                 'ratingValue':datadmp.iloc[i]['LevelOfEvidence'],
                                 'reviewAspect':'Oxford 2011 Levels of Evidence',
                                 'author':authorObject}}
        cleandata.append(tmpdict)
    return(cleandata)

In [ ]:
def update_filelist():
    all_files = os.listdir('data/tables/')
    updatefiles = all_files.remove('COVID-19 LST Data.csv')
    initial_file = 'data/tables/COVID-19 LST Data.csv'
    df = read_csv(initial_file,header=0,usecols=['PMID','Topics','LevelOfEvidence','Methodology','Updated Date'])
    if updatefiles!=None:
        for eachfile in updatefiles:
            tmpfile = read_csv('data/tables/'+eachfile,header=0,usecols=['PMID','Topics','LevelOfEvidence','Methodology','Updated Date'])
            df = pandas.concat((df,tmpfile),ignore_index=True)
    else:
        nochange=True
    df.sort_values('Updated Date',ascending=False,inplace=True)
    df.drop_duplicates(subset='PMID',keep='first',inplace=True)
    return(df)    

In [ ]:
## Run an update

def run_loe_update():
    datadmp = update_filelist()
    dictlist = generate_lst_dump(datadmp)
    with open('results/update_dumps/litcovid_lst.json', 'w', encoding='utf-8') as f:
        f.write(json.dumps(dictlist, indent=4))
       

## Parse PDFs for initial dump

In [ ]:
def generate_abstract(publist):
    separator = ', '
    abstract = "Analytical reviews on the level of evidence presented in publications. This report specifically covers the following publications: "+ separator.join(publist)
    return(abstract)

In [ ]:
### Batch convert DOIs
def convert_dois(doilist):
    doistring = '"' + '","'.join(doilist) + '"'
    r = requests.post("https://api.outbreak.info/resources/query/", params = {'q': doistring, 'scopes': 'doi', 'fields': '_id,name,url,doi'})
    if r.status_code == 200:
        rawresult = pandas.read_json(r.text)
        cleanresult = rawresult.drop_duplicates(subset='_id',keep="first")
        if len(doilist)>len(cleanresult):
            missing = [x for x in doilist if x not in cleanresult['doi'].unique().tolist()]
        else:
            missing = None
        cleanresult.drop(columns=['doi','_score','query'],inplace=True)
    return(cleanresult, missing)

### Convert a single doi
##"https://api.outbreak.info/resources/query?q=doi:"+doi

### Batch fetch pmid meta
def get_pmid_meta(pmidlist):
    pmidstring = '"' + '","'.join(pmidlist) + '"'
    r = requests.post("https://api.outbreak.info/resources/query/", params = {'q': pmidstring, 'scopes': '_id', 'fields': '_id,name,url'})
    if r.status_code == 200:
        rawresult = pandas.read_json(r.text)
        cleanresult = rawresult[['_id','name','url']].loc[rawresult['_score']==1].copy()
        cleanresult.drop_duplicates(subset='_id',keep="first", inplace=True)
        if len(pmidlist)>len(cleanresult):
            missing = [x for x in pmidlist if x not in cleanresult['_id'].unique().tolist()]
        else:
            missing = None
    return(cleanresult, missing)    


In [ ]:
def parse_pdf(eachfile):
    pdffile = open('data/reports/'+eachfile,'rb')
    pdf = pypdf.PdfFileReader(pdffile)
    pages = pdf.getNumPages()
    key = '/Annots'
    uri = '/URI'
    ank = '/A'
    allurls = []
    for page in range(pages):
        pageSliced = pdf.getPage(page)
        pageObject = pageSliced.getObject()
        if key in pageObject:
            ann = pageObject[key]
            for a in ann:
                u = a.getObject()
                if ank in u:
                    if uri in u[ank]:
                        allurls.append(u[ank][uri])
    pmidlist = []
    doilist = []
    for eachurl in allurls:
        if 'pubmed' in eachurl and '?' not in eachurl:
            pmid = eachurl.replace("https://www.ncbi.nlm.nih.gov/pubmed/","").replace("https://pubmed.ncbi.nlm.nih.gov/","").rstrip("/")
            if "#affiliation" in pmid:
                trupmid = pmid.split("/")[0]
                tmpid = 'pmid'+trupmid
            else:
                tmpid = 'pmid'+pmid
            pmidlist.append(tmpid)
        elif 'doi' in eachurl:
            doi = eachurl.replace("https://doi.org/","").rstrip("/")
            doilist.append(doi)
        pmidlist = list(set(pmidlist))
        doilist = list(set(doilist))
    return(pmidlist,doilist)

def merge_meta(pmidlist,doilist):
    if len(doilist)>0:
        doianns,missing_dois = convert_dois(doilist)
        doicheck = True
    else:
        doicheck = False
        missing_dois = None
    if len(pmidlist)>0:
        pmidanns,missing_pmids = get_pmid_meta(pmidlist)
        pmidcheck = True
    else:
        pmidcheck = False
        missing_pmids = None
    if doicheck==True and pmidcheck==True:
        basedOndf = pandas.concat((pmidanns,doianns),ignore_index=True)
    elif doicheck==True and pmidcheck==False:
        basedOndf = doianns
    elif doicheck==False and pmidcheck==True:
        basedOndf = pmidanns     
    if missing_pmids!=None and missing_dois!=None:
        missing = list(set(missing_pmids).union(set(missing_dois)))
    elif missing_pmids==None and missing_dois!=None:
        missing = missing_dois
    elif missing_pmids!=None and missing_dois==None:
        missing = missing_pmids
    else:
        missing = None
    return(basedOndf,missing)

In [ ]:
def save_missing(missing):
    try:
        missing_list = pickle.load(open('results/pubs_not_yet_in_outbreak.txt','rb'))
        if missing != None:
            total_missing = list(set([*missing_list, *missing]))
            with open('results/pubs_not_yet_in_outbreak.txt','wb') as dmpfile:
                pickle.dump(total_missing,dmpfile)
    except:
        if missing != None:
            with open('results/pubs_not_yet_in_outbreak.txt','wb') as dmpfile:
                pickle.dump(total_missing,dmpfile)

In [ ]:
def generated_citedBy_dict():
    txtdmp = read_csv('results/update dumps/litcovid_citedBy.tsv', delimiter='\t', header=0, index_col=0)
    dictlist = []
    for i in range(len(txtdump)):
        tmpdict={'_id':txtdump.iloc[i]['_id'],'citedBy':[{'@type':'Publication',
                                                                'identifier':txtdump.iloc[i]['identifier'],
                                                                'name':txtdump.iloc[i]['name'],
                                                                'url':txtdump.iloc[i]['url']}]}
        dictlist.append(tmpdict)
    with open('results/update dumps/litcovid_citedBy.json', 'w', encoding='utf-8') as f:
        json.dump(dictlist, f)


In [ ]:
## Note that strftime("%d") will give the day with a leading zero
## In windows, strftime("%#d") will give it without leading zeros
## In linux, strftime("%-d") will give it without leading zeros
def generate_report_url(datePublished):
    urlbase = "https://www.covid19lst.org/post/"
    urlend = "daily-covid-19-lst-report"
    is_windows = sys.platform.startswith('win')
    if is_windows==True:
        reporturl = urlbase+datePublished.strftime("%B").lower()+"-"+datePublished.strftime("%#d")+"-"+urlend
    else:
        reporturl = urlbase+datePublished.strftime("%B").lower()+"-"+datePublished.strftime("%-d")+"-"+urlend
    return(reporturl)

In [ ]:
def generate_report_meta(filelist):
    report_pmid_df = pandas.DataFrame(columns=['_id','name','identifier','url'])
    curatedByObject = generate_curator()
    author = generate_author()
    for eachfile in filelist:
        reportdate = eachfile[0:4]+'.'+eachfile[4:6]+'.'+eachfile[6:8]
        datePublished = datetime.fromisoformat(eachfile[0:4]+'-'+eachfile[4:6]+'-'+eachfile[6:8])
        name = "Covid-19 LST Report "+reportdate
        reporturl = generate_report_url(datePublished)
        report_id = 'lst'+reportdate
        pmidlist,doilist = parse_pdf(eachfile)
        basedOndf,missing = merge_meta(pmidlist,doilist)
        reportlinkdf = basedOndf[['_id','url']]
        reportlinkdf['identifier']=report_id
        reportlinkdf['url']=reporturl
        reportlinkdf['name']=name
        report_pmid_df = pandas.concat(([report_pmid_df,reportlinkdf]),ignore_index=True)
        report_pmid_df.drop_duplicates(keep='first',inplace=True)
        report_pmid_df.to_csv('data/report_pmid_df.txt',sep='\t',header=True)
        save_missing(missing)
        abstract = generate_abstract(basedOndf['_id'].unique().tolist())
        metadict = {"@context": {"schema": "http://schema.org/", "outbreak": "https://discovery.biothings.io/view/outbreak/"}, 
                    "@type": "Publication", "journalName": "COVID-19 LST Daily Summary Reports", "journalNameAbbreviation": "covid19LST", 
                    "publicationType": "Review", "license":"(CC BY-NC-SA 4.0) (http://creativecommons.org/licenses/by-nc-sa/4.0/)",
                    "_id":report_id,"curatedBy": curatedByObject,"abstract": abstract, "name": name, 
                    "datePublished": datePublished.strftime("%y-%m-%d"),"url": reporturl,"author":[author], 
                    "isBasedOn":basedOndf.to_dict('records')}
        yield(metadict)

In [ ]:
dumpdir = 'data/reports/'
filelist = os.listdir(dumpdir)
testlist = filelist[0:2]
metadict = generate_report_meta(testlist)
for eachdict in metadict:
    print(eachdict)

## Download PDFs from google drive

https://drive.google.com/drive/folders/1603ahBNdt1SnSaYYBE-G8SA6qgRTQ6fF

but only if they were uploaded after a specific date. The reason for this is that prior to 2020.09.11, there was variability in the file naming and file names were sometimes incorrect (202080 instead of 202008). These have been manually corrected in the initial dump, but are otherwise still incorrect in the google drive.

Note that this script uses the googledrive api which requires authentification even when accessing a public google drive. To fulfill this requirement without needing to manually log in, credentials from a service account are needed  The googledrive API is only used to read the files in the drive so that the newest ones (by date) can be identified, and their id's taken.

Additionally the the pydrive2 library (use to access the google drive api) sometimes has trouble finding the client_secrets.json file, so you may need to manually point to it.

The downloader uses the GoogleDriveDownloader library which is based off of requests and should not require the google drive api.

In [ ]:
## This function identifies files uploaded after 2020.09.11 that have NOT yet been downloaded
## Note that this is the function if a service account is not available. It requires a login

def check_google():
    from pydrive2.auth import GoogleAuth
    from pydrive2.drive import GoogleDrive

    GoogleAuth.DEFAULT_SETTINGS['client_config_file'] = 'client_secrets.json' ##point to secrets file location
    gauth = GoogleAuth()
    #gauth.LocalWebserverAuth()

    drive = GoogleDrive(gauth)
    file_id = '1603ahBNdt1SnSaYYBE-G8SA6qgRTQ6fF'
    file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % file_id}).GetList()
    
    df = pandas.DataFrame(file_list)
    dfclean = df[['createdDate','id','title']].copy()
    dfclean['date'] = pandas.to_datetime(dfclean['createdDate'],format='%Y-%m-%d', errors='coerce')
    lastupdate = dfclean.loc[dfclean['createdDate']=='2020-09-11T01:53:29.639Z'].iloc[0]['date']
    dfnew = dfclean.loc[dfclean['date']>lastupdate]
    
    all_files = os.listdir('data/reports/')
    new_files = [item  for item in all_files if item not in dfnew['title'].unique().tolist()]
    reportdf = dfnew.loc[dfnew['title'].isin(new_files)]
    return(reportdf)

In [3]:
## This function identifies files uploaded after 2020.09.11 that have NOT yet been downloaded
## Note that this is the function if a service account IS available. 
def check_google():
    from pydrive2.auth import GoogleAuth
    from pydrive2.drive import GoogleDrive
    from pydrive2.auth import ServiceAccountCredentials
    
    gauth = GoogleAuth()
    scope = ['https://www.googleapis.com/auth/drive']
    gauth.credentials = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
    drive = GoogleDrive(gauth)
    file_id = '1603ahBNdt1SnSaYYBE-G8SA6qgRTQ6fF'
    file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % file_id}).GetList()
    
    df = pandas.DataFrame(file_list)
    dfclean = df[['createdDate','id','title']].copy()
    dfclean['date'] = pandas.to_datetime(dfclean['createdDate'],format='%Y-%m-%d', errors='coerce')
    lastupdate = dfclean.loc[dfclean['createdDate']=='2020-09-11T01:53:29.639Z'].iloc[0]['date']
    dfnew = dfclean.loc[dfclean['date']>lastupdate]
    
    all_files = os.listdir('data/reports/')
    new_files = [item  for item in all_files if item not in dfnew['title'].unique().tolist()]
    reportdf = dfnew.loc[dfnew['title'].isin(new_files)]
    return(reportdf)

         kind                                 id                  etag  \
0  drive#file  1EyccnGO2UOun668cv4ddE6kg4jggm31J  "MTYwMDQ0OTE5MTAwMA"   
1  drive#file  1fxpNQRGhg8UNBIzSEBDKD0m-DjSB-1fE  "MTYwMDM5NDk3MDAwMA"   

                                            selfLink  \
0  https://www.googleapis.com/drive/v2/files/1Eyc...   
1  https://www.googleapis.com/drive/v2/files/1fxp...   

                                      webContentLink  \
0  https://drive.google.com/uc?id=1EyccnGO2UOun66...   
1  https://drive.google.com/uc?id=1fxpNQRGhg8UNBI...   

                                       alternateLink  \
0  https://drive.google.com/file/d/1EyccnGO2UOun6...   
1  https://drive.google.com/file/d/1fxpNQRGhg8UNB...   

                                           embedLink  \
0  https://drive.google.com/file/d/1EyccnGO2UOun6...   
1  https://drive.google.com/file/d/1fxpNQRGhg8UNB...   

                                            iconLink  \
0  https://drive-thirdparty.googleusercontent

In [ ]:
## This is the function to actually conduct the download
def download_reports(reportdf):
    from google_drive_downloader import GoogleDriveDownloader as gdd
    for i in range(len(dfnew)):
        title = reportdf.iloc[i]['title']
        eachid = reportdf.iloc[i]['id']
        gdd.download_file_from_google_drive(file_id=eachid,
                                            dest_path='data/reports/'+title,
                                            unzip=False)    

In [ ]:
## Run the functions
reportdf = check_google()
download_reports(reportdf)

## Download CSV data updates

In [ ]:
#### load list of files that have already been run/loaded
def load_prior_runs():
    prior_runs = pickle.load(open('results/prior_runs.txt','rb'))
    return(prior_runs)    

In [ ]:
#### Checks folder for new data dump to parse against list that have already been run/loaded
def update_check():
    prior_runs = load_prior_runs()
    all_files = os.listdir('data/reports/')
    new_files = [item  for item in all_files if item not in prior_runs]
    return(new_files)

In [ ]:
#### Generates list of PMIDs to update with COVID-19 LST info

In [ ]:
#### parse into json with covid-19 LST info to add to existing pmid data and dump to results folder

## Order of operations
1. Generate the reports
2. Generate the citedBy dump to be appended
3. Generate the litcovid extra value info

In [ ]:
##### Unit test
##https://www.covid19lst.org/post/september-11-daily-covid-19-lst-report
#dt.strftime("%A, %d. %B %Y %I:%M%p")
#'Tuesday, 21. November 2006 04:30PM'

report_pmid_df = pandas.DataFrame(columns=['_id','name','identifier','url'])
curatedByObject = generate_curator()
author = curatedByObject.copy()
author.pop("curationDate")
curatedByObject.pop('members')

for eachfile in filelist[103:104]:
    reportdate = eachfile[0:4]+'.'+eachfile[4:6]+'.'+eachfile[6:8]
    datePublished = datetime.fromisoformat(eachfile[0:4]+'-'+eachfile[4:6]+'-'+eachfile[6:8])
    name = "Covid-19 LST Report "+reportdate
    reporturl = generate_report_url(datePublished)
    report_id = 'lst'+reportdate
    pmidlist,doilist = parse_pdf(eachfile)
    basedOndf,missing = merge_meta(pmidlist,doilist)
    reportlinkdf = basedOndf[['_id','url']]
    reportlinkdf['identifier']=report_id
    reportlinkdf['url']=reporturl
    reportlinkdf['name']=name
    report_pmid_df = pandas.concat(([report_pmid_df,reportlinkdf]),ignore_index=True)
    report_pmid_df.drop_duplicates(keep='first',inplace=True)
    report_pmid_df.to_csv('data/report_pmid_df.txt',sep='\t',header=True)
    save_missing(missing)
    abstract = generate_abstract(basedOndf['_id'].unique().tolist())
    metadict = {"@context": {"schema": "http://schema.org/", "outbreak": "https://discovery.biothings.io/view/outbreak/"}, 
                "@type": "Publication", "journalName": "COVID-19 LST Daily Summary Reports", "journalNameAbbreviation": "covid19LST", 
                "publicationType": "Review", "license":"(CC BY-NC-SA 4.0) (http://creativecommons.org/licenses/by-nc-sa/4.0/)",
                "_id":report_id,"curatedBy": curatedByObject,"abstract": abstract, "name": name, 
                "datePublished": datePublished.strftime("%y-%m-%d"),"url": reporturl, 
                "isBasedOn":basedOndf.to_dict('records')}
    print(metadict)